In [28]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.pyplot as plt


n_qubits = 4

qubits = cirq.GridQubit.rect(1, n_qubits)
g=1
J=1.21
observable = [-float(0.5*g)*cirq.Z.on(q) for q in qubits] 
for q in range(len(qubits)):
    observable.append(-float(0.5*J)*cirq.X.on(qubits[q])*cirq.X.on(qubits[(q+1)%len(qubits)]))



gate=[cirq.rz, cirq.rx, cirq.rz]
for k in range(25):
    circuit = []
    symbols=[]
    for k in range(len(qubits)):
        for j in range(3):
            symbol="th_"+str(k)+"_"+str(j)
            symbols.append(symbol)
            circuit.append(gate[j](sympy.Symbol(symbol)).on(qubits[k]))
    #for k in range(len(qubits)):
    #    circuit.append(cirq.CNOT(qubits[k], qubits[(k+1)%len(qubits)]))
    circuit = cirq.Circuit(circuit)

    circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
    output = tfq.layers.Expectation(backend=cirq.DensityMatrixSimulator(noise=cirq.depolarize(0.00001)))(
    circuit_input,
    symbol_names=symbols,
    operators=tfq.convert_to_tensor([observable]),
    initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

    model = tf.keras.Model(inputs=circuit_input, outputs=output)
    adam = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=adam, loss='mse')

    tfqcircuit = tfq.convert_to_tensor([circuit])
    qoutput = tf.ones((1, 1))*-2.*n_qubits
    
    model.set_weights([tf.random.uniform((len(symbols),))])
    model.fit(x=tfqcircuit, y=qoutput,validation_steps=None,
              steps_per_epoch=1, batch_size=1, epochs=20, verbose=1, 
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

Train on 1 samples
Epoch 1/20
1/1 [==============================] - 2s 2s/sample - loss: 60.3826
Epoch 2/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3699
Epoch 3/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3575
Epoch 4/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3455
Epoch 5/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3337
Epoch 6/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3224
Epoch 7/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3113
Epoch 8/20
1/1 [==============================] - 1s 1s/sample - loss: 60.3006
Epoch 9/20
1/1 [==============================] - 1s 1s/sample - loss: 60.2901
Epoch 10/20
1/1 [==============================] - 1s 1s/sample - loss: 60.2798
Epoch 11/20
1/1 [==============================] - 1s 1s/sample - loss: 60.2696
Epoch 12/20
1/1 [==============================] - 1s 1s/sample - loss: 60.2595
Epoch 13/20
1/1 [=============

In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))

In [ ]:
circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
output = tfq.layers.Expectation()(
circuit_input,
symbol_names=symbols,
operators=tfq.convert_to_tensor([observable]),
initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!

model = tf.keras.Model(inputs=circuit_input, outputs=output)
adam = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=adam, loss='mse')
tfqcircuit = tfq.convert_to_tensor([circuit])
qoutput = tf.ones((1, 1))*-2.*n_qubits
model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=250, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)])

In [ ]:
np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))